In [3]:
import rasterio
from rasterio.plot import show
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# First, let's load the raster file using rasterio
def plot_conservation_tif(tif_path="output/conservation_solution.tif"):
    """
    Create an interactive plot of conservation priorities from a GeoTIFF file
    
    Parameters:
    -----------
    tif_path : str
        Path to the conservation solution GeoTIFF file
    """
    # Open the raster file
    with rasterio.open(tif_path) as src:
        # Read the raster data
        raster_data = src.read(1)  # Read the first band
        
        # Get metadata
        height, width = raster_data.shape
        bounds = src.bounds
        transform = src.transform
        crs = src.crs
        
        # Print basic information about the raster
        print(f"Raster dimensions: {width} x {height}")
        print(f"Coordinate system: {crs.to_string()}")
        print(f"Bounding box: {bounds}")
        
        # Count number of conservation areas (cells with value 1)
        selected_cells = np.sum(raster_data == 1)
        total_cells = width * height
        pct_selected = (selected_cells / total_cells) * 100
        
        print(f"Conservation priorities summary:")
        print(f"  - Selected cells: {selected_cells:,} ({pct_selected:.2f}%)")
        print(f"  - Total planning units: {total_cells:,}")
        
        # Create a custom colormap for conservation planning
        # 0 = Not selected (light gray), 1 = Selected (dark green)
        colors = ["#f0f0f0", "#006d2c"]
        colormap = ListedColormap(colors)
        
        # Create a figure with two subplots
        fig = make_subplots(
            rows=1, cols=2,
            column_widths=[0.7, 0.3],
            specs=[[{"type": "mapbox"}, {"type": "xy"}]],
            subplot_titles=("Conservation Priority Areas", "Solution Statistics")
        )
        
        # Get coordinates for mapping
        # These calculations transform pixel coordinates to geographic coordinates
        lon = np.linspace(bounds.left, bounds.right, width)
        lat = np.linspace(bounds.top, bounds.bottom, height)
        lon_grid, lat_grid = np.meshgrid(lon, lat)
        
        # Add the heatmap to the first subplot
        fig.add_trace(
            go.Densitymapbox(
                lat=lat_grid.flatten(),
                lon=lon_grid.flatten(),
                z=raster_data.flatten(),
                radius=5,  # Adjust this based on your data resolution
                colorscale=[[0, colors[0]], [1, colors[1]]],
                zmin=0,
                zmax=1,
                opacity=0.75,
                showscale=False,
                hovertemplate='Status: %{z}<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Add a bar chart showing the number of selected vs not selected cells
        fig.add_trace(
            go.Bar(
                x=['Not Selected', 'Selected'],
                y=[total_cells - selected_cells, selected_cells],
                marker_color=['#f0f0f0', '#006d2c'],
                text=[f"{total_cells - selected_cells:,}", f"{selected_cells:,}"],
                textposition='auto',
                hoverinfo='y+text',
                name='Cell Count'
            ),
            row=1, col=2
        )
        
        # Add a percentage indicator
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=pct_selected,
                title={'text': "Percent Selected"},
                gauge={
                    'axis': {'range': [0, 100]},
                    'bar': {'color': "#006d2c"},
                    'steps': [
                        {'range': [0, 100], 'color': "#f0f0f0"}
                    ]
                },
                domain={'row': 0, 'column': 1, 'y': [0, 0.5]}
            ),
            row=1, col=2
        )
        
        # Update the layout
        fig.update_layout(
            title_text='Conservation Priority Areas in Colorado',
            title_x=0.5,  # Center the title
            height=700,
            width=1100,
            mapbox=dict(
                style="carto-positron",  # Clean, light basemap
                zoom=7,  # Adjust zoom level to fit your study area
                center={"lat": 39.0, "lon": -105.5},  # Center on Colorado
                layers=[
                    {
                        "below": 'traces',
                        "sourcetype": "raster",
                        "sourceattribution": "USGS National Map",
                        "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]
                    }
                ]
            ),
            showlegend=False
        )
        
        # Add an annotation explaining how to interact with the map
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.01, y=0.01,
            text="Hover for cell status • Drag to pan • Scroll to zoom",
            showarrow=False,
            font=dict(size=10),
            bgcolor="white",
            borderpad=4,
            opacity=0.8
        )
        
        # Show the figure
        return fig

# Create a secondary function to produce a simpler visualization
# if the densitymapbox approach is too slow for large rasters
def plot_conservation_tif_simple(tif_path="output/conservation_solution.tif"):
    """
    Create a simpler but faster plot for large GeoTIFF files
    """
    # Open the raster file
    with rasterio.open(tif_path) as src:
        raster_data = src.read(1)
        bounds = src.bounds
        
        # Create a simple matplotlib plot and save it as an image
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Create a custom colormap
        colors = ["#f0f0f0", "#006d2c"]  # Light gray for 0, dark green for 1
        cmap = ListedColormap(colors)
        
        # Plot the raster with the custom colormap
        show(raster_data, ax=ax, cmap=cmap, transform=src.transform)
        
        # Add a title
        ax.set_title('Conservation Priority Areas')
        
        # Save the plot as an image
        plt.savefig('conservation_map.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Now create a plotly figure with the image
        fig = go.Figure()
        
        # Add the image
        fig.add_layout_image(
            dict(
                source='conservation_map.png',
                x=0,
                y=1,
                xref="paper",
                yref="paper",
                sizex=1,
                sizey=1,
                sizing="stretch",
                opacity=1,
                layer="below"
            )
        )
        
        # Add some text information
        selected_cells = np.sum(raster_data == 1)
        total_cells = raster_data.size
        pct_selected = (selected_cells / total_cells) * 100
        
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.5, y=0.02,
            text=f"Selected: {selected_cells:,} cells ({pct_selected:.2f}% of total)",
            showarrow=False,
            font=dict(size=14),
            bgcolor="white",
            borderpad=4,
            opacity=0.8
        )
        
        # Update layout
        fig.update_layout(
            width=800,
            height=650,
            margin=dict(l=0, r=0, t=0, b=0)
        )
        
        # Make it interactive with hover info
        fig.update_layout(
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            )
        )
        
        return fig

# Example usage in a Jupyter notebook:
# ----------------------------------
# Import the functions defined above
# 
# # Try the advanced visualization first
# try:
fig = plot_conservation_tif("../R-data/output/conservation_solution.tif")
fig.show()
# except Exception as e:
#     print(f"Advanced visualization failed: {e}")
#     print("Falling back to simpler visualization...")
#     fig2 = plot_conservation_tif_simple("output/conservation_solution.tif")
#     fig2.show()

   places_fmv_all                                           geometry
0               0  MULTIPOLYGON (((-9455666.41586 4550060.53593, ...
1               1  MULTIPOLYGON (((-9008282.61012 4558948.96287, ...
Number of planning units: 2


KeyError: 'selected'